# VI: First Practical Work

**Authors:** Gerard Comas & Marc Franquesa.

## Data Processing
Processing all datasets in this notebook

In [ ]:
# Initial imports
import pandas as pd
import numpy as np
import altair as alt
import geopandas as gpd
import warnings
from shapely.geometry import shape, Point

warnings.simplefilter(action='ignore', category=FutureWarning)

### Collisions dataset

In [ ]:
collisions = pd.read_csv("./original-data/collisions.csv")

print(collisions.columns)
print(f"Initial amount of rows: {len(collisions)}")

In [ ]:
# Adding a CRASH DATETIME column as well as several checks to make sure we have the correct dataset

# Truncating to the hour because some (most) rows are already truncated and we don't need more information
collisions["CRASH DATETIME"] = pd.to_datetime(collisions["CRASH DATE"] + " " + collisions["CRASH TIME"]).dt.floor("H")

# Adding day of week column
collisions["CRASH WEEKDAY"] = collisions["CRASH DATETIME"].dt.day_name()

# Adding BEFORE COVID column
collisions["AFTER COVID"] = collisions['CRASH DATETIME'].dt.year == 2020

print(f"First crash: {collisions['CRASH DATETIME'].sort_values().iloc[0]}")

print(f"Last crash of 2018: {collisions[collisions['CRASH DATETIME'].dt.year == 2018]['CRASH DATETIME'].sort_values().iloc[-1]}")

print(f"First crash of 2020: {collisions[collisions['CRASH DATETIME'].dt.year == 2020]['CRASH DATETIME'].sort_values().iloc[0]}")

print(f"Last crash: {collisions['CRASH DATETIME'].sort_values().iloc[-1]}")

print(f"Collisions in 2019: {len(collisions[collisions['CRASH DATETIME'].dt.year == 2019])}")

In [ ]:
# Checking if LOCATION contains the same information as LATITUDE and LONGITUDE
# We will take advantage of the fact that if a value is NaN in python then
# value == value will return False
def same_information():
    location = collisions["LOCATION"].tolist()
    lat, lon = collisions["LATITUDE"].tolist(), collisions["LONGITUDE"].tolist()
    for i, row in enumerate(location):
        # LOCATION is not NaN
        if row == row:
            if not list(map(float, row[1: -1].split(", "))) == [lat[i], lon[i]]: return False
        # LOCATION is NaN
        else:
            # If lat or lon is different to Nan return False
            if lat[i] == lat[i] or lon[i] == lon[i]: return False
    return True

print(same_information())

In [ ]:
# Column selection
cols = [
    "CRASH DATETIME",
    "CRASH WEEKDAY",
    "AFTER COVID",
    "BOROUGH",
    "LATITUDE",
    "LONGITUDE",
    "NUMBER OF PERSONS INJURED",
    "NUMBER OF PERSONS KILLED",
    "VEHICLE TYPE CODE 1",
]
collisions = collisions[cols]

# Number of missing values in each column
print(collisions.isnull().sum())

In [ ]:
# Fill in missing values with 0 for the injured/killed columns
collisions["NUMBER OF PERSONS INJURED"].fillna(0, inplace=True)
collisions["NUMBER OF PERSONS KILLED"].fillna(0, inplace=True)

We will now classify all vehicle types into these categories:
* Bicycle
* Car
* E-bike
* E-scooter
* Truck
* Bus
* Motorcycle
* Other
* Unknown

In [ ]:
classified_vehicles = {
    "Station Wagon/Sport Utility Vehicle": "Car",
    "Sedan": "Car",
    "Bus": "Bus",
    "Tractor Truck Diesel": "Truck",
    "Taxi": "Car",
    "E-Scooter": "E-scooter",
    "Flat Bed": "Truck",
    "Motorbike": "Motorcycle",
    "Motorcycle": "Motorcycle",
    "Box Truck": "Truck",
    "Pick-up Truck": "Truck",
    "Bike": "Bicycle",
    "Dump": "Truck",
    "Concrete Mixer": "Truck",
    "Van": "Truck",
    "PK": "Other",
    "Golf Cart": "Other",
    "LIMO": "Car",
    "Tanker": "Truck",
    "AMBULANCE": "Ambulance",
    "Convertible": "Car",
    "E-Bike": "E-bike",
    "Moped": "Motorcycle",
    "Fire Truck": "Truck",
    "nan": "Other",
    "Tractor Truck Gasoline": "Truck",
    "Ambulance": "Ambulance",
    "forlift": "Other",
    "MOTOR SKAT": "Other",
    "FDNY LADDE": "Other",
    "Tow Truck / Wrecker": "Truck",
    "FIRE TRUCK": "Truck",
    "PICK UP": "Other",
    "Garbage or Refuse": "Truck",
    "GARBAGE TR": "Truck",
    "Chassis Cab": "Truck",
    "Bulk Agriculture": "Other",
    "Can": "Other",
    "van": "Truck",
    "Carry All": "Other",
    "FLATBED FR": "Truck",
    "Open Body": "Other",
    "4 dr sedan": "Car",
    "Motorscooter": "Motorcycle",
    "Minibike": "Motorcycle",
    "Flat Rack": "Other",
    "Armored Truck": "Truck",
    "School Bus": "Bus",
    "FDNY TRUCK": "Truck",
    "truck": "Truck",
    "UNK": "Unknown",
    "TRAILER": "Other",
    "FIRTRUCK": "Truck",
    "MOPED": "Motorcycle",
    "Lift Boom": "Other",
    "fdny ems": "Other",
    "AMBULACE": "Ambulance",
    "bus": "Bus",
    "BOX TRUCK": "Truck",
    "Street Swe": "Other",
    "Scooter": "Motorcycle",
    "FDNY fire": "Other",
    "DELIVERY": "Other",
    "Cement Tru": "Truck",
    "USPS/GOVT": "Other",
    "Pedicab": "Other",
    "TRUCK VAN": "Truck",
    "UTILITY": "Other",
    "Pick up tr": "Other",
    "UNKNOWN": "Unknown",
    "Multi-Wheeled Vehicle": "Other",
    "SUV": "Car",
    "utility": "Other",
    "POWER SHOV": "Other",
    "DELIVERY T": "Other",
    "SWT": "Other",
    "Trac": "Other",
    "FDNY AMBUL": "Ambulance",
    "AMBU": "Other",
    "USPS": "Other",
    "FLAT": "Other",
    "Beverage Truck": "Truck",
    "E-BIKE": "E-bike",
    "3-Door": "Car",
    "Fork Lift": "Other",
    "Refrigerated Van": "Truck",
    "PSD": "Other",
    "Fire Engin": "Other",
    "FORKLIFT": "Other",
    "TRAC": "Other",
    "Tow Truck": "Truck",
    "COURIER": "Other",
    "Courier": "Other",
    "Leased amb": "Other",
    "SMART CAR": "Car",
    "message si": "Other",
    "scooter": "Motorcycle",
    "E-UNICYCLE": "E-scooter",
    "Street Cle": "Other",
    "box": "Other",
    "F550": "Truck",
    "DELV": "Other",
    "SKATEBOARD": "Other",
    "Lawnmower": "Other",
    "almbulance": "Other",
    "dark color": "Other",
    "Work Van": "Other",
    "ford van": "Truck",
    "ambulance": "Ambulance",
    "Fire truck": "Truck",
    "Minicycle": "Motorcycle",
    "PC": "Other",
    "box truck": "Truck",
    "FDNY ENGIN": "Other",
    "commercial": "Other",
    "Unknown": "Unknown",
    "Tractor tr": "Truck",
    "2 dr sedan": "Car",
    "FD LADDER": "Other",
    "abulance": "Other",
    "FDNY Engin": "Other",
    "OTH": "Other",
    "Go kart": "Other",
    "Trailer": "Other",
    "TRUCK": "Truck",
    "Stake or Rack": "Other",
    "COMMERCIAL": "Other",
    "CHEVY EXPR": "Other",
    "SLINGSHOT": "Other",
    "dilevery t": "Other",
    "FDNY #226": "Other",
    "FREIGHT FL": "Other",
    "Fork lift": "Other",
    "UTIL": "Other",
    "UNKN": "Other",
    "FDNY FIRE": "Other",
    "ELECTRIC S": "Other",
    "FIRETRUCK": "Truck",
    "MOVING VAN": "Truck",
    "usps": "Other",
    "moped": "Motorcycle",
    "forklift": "Other",
    "UPS TRUCK": "Truck",
    "backhoe": "Other",
    "Delv": "Other",
    "dump truck": "Truck",
    "Freight": "Other",
    "Horse": "Other",
    "Cargo Van": "Truck",
    "USPS VAN": "Other",
    "TRUCK FLAT": "Truck",
    "BOBCAT FOR": "Other",
    "Tractor Tr": "Truck",
    "Pumper": "Other",
    "DELIVERY V": "Other",
    "DOT EQUIPM": "Other",
    "fire truck": "Truck",
    "Livestock Rack": "Other",
    "GEN  AMBUL": "Ambulance",
    "J1": "Other",
    "DUMP": "Other",
    "18 WHEELER": "Truck",
    "MAIL TRUCK": "Other",
    "UTILITY VE": "Other",
    "MOTORSCOOT": "Motorcycle",
    "government": "Other",
    "trailer": "Other",
    "FIRE ENGIN": "Other",
    "Front-Load": "Other",
    "DRILL RIG": "Other",
    "SCOOTER": "Motorcycle",
    "Wh Ford co": "Other",
    "suburban": "Car",
    "E REVEL SC": "Other",
    "ROAD SWEEP": "Other",
    "LIGHT TRAI": "Other",
    "Tractor": "Truck",
    "UT": "Other",
    "USPS TRUCK": "Other",
    "cross": "Other",
    "Van Camper": "Other",
    "AMBULENCE": "Ambulance",
    "FOOD TRUCK": "Other",
    "Bucket Tru": "Other",
    "gator": "Other",
    "FDNY Ambul": "Ambulance",
    "JOHN DEERE": "Other",
    "f-250": "Other",
    "MECHANICAL": "Other",
    "WORK VAN": "Other",
    "NYC FD": "Other",
    "MTA BUS": "Bus",
    "NYC AMBULA": "Ambulance",
    "GOLF CART": "Other",
    "FLATBED": "Truck",
    "Trc": "Other",
    "FORK LIFT": "Other",
    "Pick up Tr": "Other",
    "postal bus": "Bus",
    "F150XL PIC": "Other",
    "ambu": "Other",
    "Pick up": "Other",
    "CAT": "Other",
    "ELEC. UNIC": "E-scooter",
    "1C": "Other",
    "SCOOT": "Motorcycle",
    "FREIG": "Other",
    "AMBUL": "Ambulance",
    "VAN T": "Other",
    "MINI": "Other",
    "Garba": "Other",
    "motor": "Other",
    "Lunch Wagon": "Other",
    "E-Bik": "E-bike",
    "Ambul": "Ambulance",
    "FDNY": "Other",
    "SCHOO": "Other",
    "Comm": "Other",
    "Fire": "Other",
    "Sanit": "Other",
    "mail": "Other",
    "RV": "Other",
    "GARBA": "Other",
    "ambul": "Ambulance",
    "FIRET": "Other",
    "FIRE": "Other",
    "SELF": "Other",
    "STAK": "Other",
    "WORKH": "Other",
    "FORKL": "Other",
    "Tract": "Other",
    "freig": "Other",
    "DELIV": "Other",
    "trail": "Other",
    "PICKU": "Other",
    "Dumps": "Other",
    "forkl": "Other",
    "fire": "Other",
    "TRK": "Other",
    "ELECT": "Other",
    "2- to": "Other",
    "BROOM": "Other",
    "TRAIL": "Other",
    "EBIKE": "E-bike",
    "Trail": "Other",
    "Glass Rack": "Other",
    "Motorized Home": "Other",
    "US POSTAL": "Other",
    "TRT": "Other",
    "BLOCK": "Other",
    "pas": "Other",
    "COM": "Other",
    "CONCR": "Other",
    "Pallet": "Other",
    "unknown": "Unknown",
    "CHERR": "Other",
    "UTV": "Other",
    "MOTOR": "Other",
    "MTA B": "Bus",
    "TRACT": "Other",
    "NYC": "Other",
    "UHAUL": "Other",
    "scoot": "Motorcycle",
    "FED E": "Other",
    "COMME": "Other",
    "TRLR": "Other",
    "LOADE": "Other",
    "rv": "Other",
    "TOWER": "Other",
    "Pick": "Other",
    "AMB": "Other",
    "NS AM": "Other",
    "UNKNO": "Unknown",
    "NEW Y": "Other",
    "TOW T": "Other",
    "GRAY": "Other",
    "tract": "Other",
    "STREE": "Other",
    "MAIL": "Other",
    "e-bik": "E-bike",
    "unk": "Unknown",
    "box t": "Other",
    "CRANE": "Other",
    "garba": "Other",
    "Pickup with mounted Camper": "Other",
    "FRONT": "Other",
    "Sprin": "Other",
    "delv": "Other",
    "POWER": "Other",
    "Box t": "Other",
    "CAMP": "Other",
    "Enclosed Body - Removable Enclosure": "Other",
    "RGS": "Other",
    "GOVER": "Other",
    "FORK": "Other",
    "UTILI": "Other",
    "POSTO": "Other",
    "firet": "Other",
    "WORK": "Other",
    "R/V C": "Other",
    "sgws": "Other",
    "Cat 9": "Other",
    "BACKH": "Other",
    "E-MOT": "E-scooter",
    "MACK": "Other",
    "SPC": "Other",
    "fork": "Other",
    "OMR": "Other",
    "semi": "Other",
    "FORK-": "Other",
    "Wheel": "Other",
    "Utili": "Other",
    "E-BIK": "E-bike",
    "fd tr": "Other",
    "SWEEP": "Other",
    "BOX T": "Other",
    "CASE": "Other",
    "FD TR": "Other",
    "Work": "Other",
    "LIBER": "Other",
    "fdny": "Other",
    "COMB": "Other",
    "HEAVY": "Other",
    "DUMPS": "Other",
    "MTA b": "Bus",
    "Hopper": "Other",
    "R/V": "Other",
    "FOOD": "Other",
    "FD tr": "Other",
    "Spc": "Other",
    "BED T": "Other",
    "comme": "Other",
    "UPS T": "Other",
    "PAS": "Other",
    "BICYC": "Bicycle",
    "Subn": "Other",
    "WHEEL": "Other",
    "Util": "Other",
    "ACCES": "Other",
    "e sco": "E-scooter",
    "BOBCA": "Other",
    "TANK": "Other",
    "TRACK": "Other",
    "utili": "Other",
    "DEMA-": "Other",
    "tow": "Other",
    "dump": "Other",
    "Elect": "Other",
    "deliv": "Other",
    "Backh": "Other",
    "CEMEN": "Other",
    "99999": "Other",
    "BULLD": "Other",
    "seagr": "Other",
    "schoo": "Other",
    "CONST": "Other",
    "self": "Other",
    "BK": "Other",
    "Semi": "Other",
    "Scoot": "Motorcycle",
    "NYPD": "Other",
    "Taxis": "Car",
}

In [ ]:
# Changing vehicle types to classification we want to use, list is found in the 
# NYC collision dataset: ATV, bicycle, car/suv, ebike, E-scooter, truck/bus,
# motorcycle, other, unknown

collisions["ORIGINAL VEHICLE"] = collisions["VEHICLE TYPE CODE 1"].fillna("unknown")
collisions = collisions.drop(columns="VEHICLE TYPE CODE 1")

collisions["VEHICLE"] = collisions["ORIGINAL VEHICLE"].replace(classified_vehicles)

collisions["VEHICLE"].value_counts()

In [ ]:
# Replacing LATITUDE and longitude values that don't make make sense for NYC into NaNs
collisions["LATITUDE"] = collisions["LATITUDE"].where(collisions["LATITUDE"].between(38, 42))
collisions["LONGITUDE"] = collisions["LONGITUDE"].where(collisions["LONGITUDE"].between(-76, -72))

# Adding our own LOCATION column, we do know that it already exists but it was easier for us this way
# If either LATITUDE or LONGITUDE is NaN then location will be NaN
def combine_columns(row):
    if pd.notna(row["LATITUDE"]) and pd.notna(row["LONGITUDE"]):
        return [row["LATITUDE"], row["LONGITUDE"]]
    else:
        return np.nan

collisions["LOCATION"] = collisions.apply(combine_columns, axis=1)

# Dropping NaNs in LOCATION and BOROUGH, if either BOROUGH or LOCATION is not NaN we will keep the row
collisions.dropna(subset=["LOCATION", "BOROUGH"], how="all", inplace=True)

print(f"Current amount of rows: {len(collisions)}")

In [ ]:
print(collisions.isnull().sum())

In [ ]:
collisions.head()

In [ ]:
collisions.to_csv("./processed-data/collisions.csv", index=False)

### Weather dataset

Data obtained from the ASOS Network of Iowa State University, with the following link: https://mesonet.agron.iastate.edu/request/download.phtml?network=NY_ASOS. <br>

We have selected the station [NYC] NEW YORK CITY (1943-Now).


In [ ]:
# Read the csv file into a pandas DataFrame
weather_2018 = pd.read_csv("./original-data/NYC_weather_2018.csv")
weather_2020 = pd.read_csv("./original-data/NYC_weather_2020.csv")

# concatenate the two dataframes
weather = pd.concat([weather_2018, weather_2020], ignore_index=True)

# M & T represents a NaN in the dataset (found in the docs)
weather = weather.replace('M', None).replace('T', None)

# print the concatenated dataframe
print(weather.columns)

# na values in the dataframe
print(weather.isna().sum())

The following columns will be removed, as they are not deemed relevant. This may be due to the focus on summer data, where columns related to snow lack significance, or because the columns inherently possess a high number of missing values: <br>
- station
- dwpf
- drct
- alti
- gust
- skyc1
- skyc2
- skyc3
- skyc4
- sky1
- skyl2
- skyl3
- skyl4
- wxcodes
- feel
- ice_accretion_1hr
- ice_accretion_3hr
- ice_accretion_6hr
- peak_wind_gust
- peak_wind_drct
- peak_wind_time
- metar
- snowdepth

In [ ]:
cols = [
    "valid",
    "tmpf",
    "relh",
    "sknt",
    "p01i",
    "mslp",
    "vsby",
]

weather = weather[cols]

weather[cols[1:]] = weather[cols[1:]].apply(pd.to_numeric)

print(weather.isna().sum())

### The analysis will involve working with the following variables.
- **`valid`**: timestamp of the observation
- **`tmpf`**: Air Temperature in Fahrenheit, typically @ 2 meters 
- **`relh`**: Relative Humidity in %
- **`sknt`**: Wind Speed in knots 
- **`p01i`**: One hour precipitation for the period from the observation time to the time of the previous hourly precipitation reset. Values are in inches.
- **`mslp`**: Sea Level Pressure in millibar 
- **`vsby`**: Visibility in miles 

Let's visually explore the missing values using the `missingno` package.

In [ ]:
import missingno as msno

# matrix plot
msno.matrix(weather)

The function `msno.matrix(weather)` generates a nullity matrix, which is a graphical representation of the absence of data in the `weather` DataFrame. Each row in the matrix corresponds to a row in the DataFrame, and white marks indicate missing values.

By observing the graph, you can look for patterns in the missing data. For instance, if you notice that white marks tend to cluster in certain areas, it could suggest that the missing data is not randomly distributed but rather related to some variable or condition. As in the case of the `sknt` column, likely due to a failure in the wind speed sensor, we should take this into account when analyzing the data.

We can also visually see that missing values in the `tmpf` and `relh` columns are related.

Another valuable insight from this graph is that the `mslp` column may not be as interesting as initially thought, as having so many random missing values doesn't make sense.

In [ ]:
# heatmap
msno.heatmap(weather)

The `msno.heatmap(weather)` function generates a heatmap showing the correlation of missing data in the `weather` DataFrame. The values on the heatmap range from -1 to 1.

A value close to 1 indicates that the presence of a missing value in one column is strongly correlated with the presence of a missing value in another column. This could suggest that missing values in both columns are being caused by the same underlying factor.

On the other hand, a value close to -1 indicates that the presence of a missing value in one column is strongly correlated with the presence of a value in another column. This could suggest that missing values in one column are being caused by the absence of missing values in the other column.

Now, we can confirm the strong correlation between the missing values in the `tmpf` and `relh` columns, as well as the significant correlation between these columns and `vsby`.

In [ ]:
# drop the columns mslp
weather.drop('mslp', axis=1, inplace=True)

Let's determine the intervals where the wind speed sensor may have stopped functioning.

In [ ]:
# filter the weather dataframe to show only data from September 2018
weather_sep2018 = weather[(weather['valid'] >= '2018-09-01') & (weather['valid'] < '2018-09-30')]

# create a mask for missing values in sknt column
mask = weather_sep2018['sknt'].isna()

# create a group identifier for consecutive missing values
group_id = (~mask).cumsum()

# group the consecutive missing values together and count the number of missing values in each group
consecutive_missing_values = mask.groupby(group_id).sum()

# print the number of consecutive missing values and the last one in the series
print(f"Number of consecutive missing values in sknt: {consecutive_missing_values.max()}")

# obtain the first and last element with id = consecutive_missing_values.idxmax() in group_id
group_id_max = consecutive_missing_values.idxmax()

first_element_valid_sep2018 = weather_sep2018.loc[group_id.eq(group_id_max).idxmax(), 'valid']
last_element_valid_sep2018 = weather_sep2018.loc[group_id.eq(group_id_max+1).idxmax(), 'valid']

# print the valid column of the first and last element of the consecutive missing value
print(f"Valid column of the first element: {first_element_valid_sep2018}")
print(f"Valid column of the last element: {last_element_valid_sep2018}")

In [ ]:
# filter the weather dataframe to show only data from June 2020
weather_jun2020 = weather[(weather['valid'] >= '2020-06-01') & (weather['valid'] < '2020-06-30')]

# create a mask for missing values in sknt column
mask = weather_jun2020['sknt'].isna()

# create a group identifier for consecutive missing values
group_id = (~mask).cumsum()

# group the consecutive missing values together and count the number of missing values in each group
consecutive_missing_values = mask.groupby(group_id).sum()

# print the number of consecutive missing values and the last one in the series
print(f"Number of consecutive missing values in sknt: {consecutive_missing_values.max()}")

# obtain the first and last element with id = consecutive_missing_values.idxmax() in group_id
group_id_max = consecutive_missing_values.idxmax()
first_element_valid_jun2020 = weather_jun2020.loc[group_id.eq(group_id_max).idxmax(), 'valid']
last_element_valid_jun2020 = weather_jun2020.loc[group_id.eq(group_id_max+1).idxmax(), 'valid']

# print the valid column of the first and last element of the consecutive missing value
print(f"Valid column of the first element: {first_element_valid_jun2020}")
print(f"Valid column of the last element: {last_element_valid_jun2020}")

Given the identified periods of wind speed sensor malfunction, a procedure is initiated to interpolate all remaining missing values, excluding those specific intervals.

In [ ]:
# interpolate the missing values in sknt column except the ones between 2020-06-01 00:51 and 2020-06-19 19:51 and between 2018-09-12 14:21 and 2018-09-17 17:51
mask1 = (weather['valid'] >= first_element_valid_jun2020) & (weather['valid'] <= last_element_valid_jun2020)
mask2 = (weather['valid'] >= first_element_valid_sep2018) & (weather['valid'] <= last_element_valid_sep2018)
mask = ~(mask1 | mask2)
weather.loc[mask, 'sknt'] = weather.loc[mask, 'sknt'].interpolate()

In the analysis of the `p01i` column, a visualization is conducted on the values preceding missing entries. This aims to provide an approximate understanding of their magnitudes. Considering the uncertainty about sensor behavior, it is plausible that the sensor may cease recording during periods of no rainfall or excessively high rainfall amounts.

In [ ]:
# get the rows where the value in the p01i column is not missing and the value in the shifted p01i column is missing
before_missing = weather.loc[weather['p01i'].notna() & weather['p01i'].shift(1).isna()]

ch = alt.Chart(before_missing).mark_bar().encode(
    x='valid:O',
    y='p01i:Q'
)

ch.properties(
    width=800,
    height=400
).configure_axisX(labels=False)


In [ ]:
print("Statistics of weather['p01i']:\n", weather['p01i'].describe())
print("\nStatistics of before_missing['p01i']:\n", before_missing['p01i'].describe())

No specific patterns or behaviors in the sensor data have been identified. Consequently, a decision has been made to interpolate the missing values.

In [ ]:
# Interpolate the missing values in p01i column
weather['p01i'] = weather['p01i'].interpolate()

As previously mentioned, there exists a correlation between the missing values in the `vsby` column and `tmpf`. Consequently, the decision has been made to interpolate only the values that are not correlated.

In [ ]:
# Interpolate the missing valurs in vsby column
weather.loc[weather['tmpf'].notna(), 'vsby'] = weather.loc[weather['tmpf'].notna(), 'vsby'].interpolate()

# get the count of missing values in each column
weather.isna().sum()

Now, the conversion to the International System of Units will be performed:
- **tmpf**: Fahrenheit to Celsius <br>
$$ Celsius = (Fahrenheit - 32) \cdot  \frac{5}{9} $$ 
- **sknt**: Knots to km/h
$$ 1 \ knot = 1.852 \ \frac{km}{h}$$
- **p01i**: inches to cm
$$ 1 \ inch  = 2.54 \ cm $$
- **vsby**: miles to km
$$ 1 \ mile = 1.609344 \ km $$

In [ ]:
weather["tmpf"] = (weather["tmpf"] - 32) * 5/9

weather['sknt'] = weather['sknt'] * 1.852

weather['p01i'] = weather['p01i'] * 2.54

weather['vsby'] = weather['vsby'] * 1.609344


weather['valid'] = pd.to_datetime(weather['valid']).dt.floor('H')
weather_grouped = weather.groupby('valid').mean().reset_index()
print(weather_grouped.head())

In [ ]:
print(f"First crash: {weather_grouped['valid'].sort_values().iloc[0]}")

print(f"Last crash of 2018: {weather_grouped[weather_grouped['valid'].dt.year == 2018]['valid'].sort_values().iloc[-1]}")

print(f"First crash of 2020: {weather_grouped[weather_grouped['valid'].dt.year == 2020]['valid'].sort_values().iloc[0]}")

print(f"Last crash: {weather_grouped['valid'].sort_values().iloc[-1]}")

print(f"Collisions in 2019: {len(weather_grouped[weather_grouped['valid'].dt.year == 2019])}")

In [ ]:
weather.describe()

weather_grouped.describe()

In [ ]:
# Save the cleaned dataframe to a csv file
weather_grouped.to_csv("processed-data/weather.csv", index=True)

### NYC Map
Currently using [NYC community district boundaries](https://data.cityofnewyork.us/City-Government/Community-Districts/yfnk-k7r4) in a geojson format. Lets add the number of collisions in each region. We find that community districts is the perfect granularity for a choropleth map. Not too little (Boroughs) not too much (zip codes).

In [ ]:
map_data = gpd.read_file(f"./original-data/map.geojson")

collisions["DISTRICT"] = collisions["LOCATION"].apply(
    lambda x: [-1] if x != x else np.where(map_data.contains(Point(x[1], x[0])))[0]
)

collisions["DISTRICT"] = collisions["DISTRICT"].apply(lambda x: -1 if len(x) == 0 else x[0]).replace(-1, np.nan)

map_data["collision_count"] = collisions.groupby(["DISTRICT"]).size()

In [ ]:

map_data.to_file("processed-data/map.geojson", driver="GeoJSON")

## Design and implementation

**Q IDEAS:**

* Q1: basic barplot?
* Q2: slope chart
* Q3: histogram?
* Q4: basic map plot
* Q5:

---
**O IDEAS:**
* Color for vehicle type


In [ ]:
# Helpful functions

def before_covid(df: pd.DataFrame) -> pd.DataFrame:
    return df[df["AFTER COVID"] == False]

def after_covid(df: pd.DataFrame) -> pd.DataFrame:
    return df[df["AFTER COVID"] == True]

### 1. Are accidents more frequent during weekdays or weekends? Is there any difference between before COVID-19 and after?

With an ambitious goal in mind, lets first plot the total collisions of each day of the week before COVID.

In [ ]:
before_covid_day_count = before_covid(collisions).groupby(["CRASH WEEKDAY"]).size().reset_index(name="counts")

weekdayorder = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]

alt.Chart(before_covid_day_count).mark_bar().encode(
    x = alt.X("CRASH WEEKDAY:O", sort=weekdayorder, axis=alt.Axis(title="Week Day")),
    y = alt.Y("counts:Q", axis=alt.Axis(title="Collisions"))
).properties(
    width=400
)

Lets now make a grouped bar chart, separating before and after covid.

In [ ]:
days_df = collisions.groupby(["CRASH WEEKDAY", "AFTER COVID"]).size().reset_index(name="counts")

before, after, all_time = "Summer 2018 (Before Covid)", "Summer 2020 (After Covid)", "All"

days_df["MOMENT"] = np.where(days_df["AFTER COVID"], after, before)

weekdayorder = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]

opacity = 0.5

colors = {
    before: "#fdc086", # Before COVID
    after: "#7fc97f", # After COVID
    all_time: "#beaed4"
}

days_ch = alt.Chart(days_df).mark_bar(
    opacity=opacity
).encode(
   x=alt.X("CRASH WEEKDAY:O", axis=alt.Axis(labelAngle=-30, title=None), sort=weekdayorder),
   xOffset="MOMENT:O",
   y=alt.Y("counts:Q", axis=alt.Axis(title="Collisions", grid=True)),
   color=alt.Color("MOMENT:O", scale=alt.Scale(domain=list(colors.keys()), range=list(colors.values())), legend=alt.Legend(title=None))
)

days_ch

Lets now add the average of before and after covid.

In [ ]:
averages = alt.Chart(days_df).mark_rule(opacity=1).encode(
    y="mean(counts):Q",
    size=alt.value(2),
    color="MOMENT:O"
)

averages + days_ch

Lets now separate the days of the week in two categories, weekdays and weekends.

In [ ]:
weekdays = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday"]
weekends = ["Saturday", "Sunday"]

weekdays_df = days_df[days_df["CRASH WEEKDAY"].isin(weekdays)]
weekends_df = days_df[days_df["CRASH WEEKDAY"].isin(weekends)]

weekdays_ch = alt.Chart(weekdays_df).mark_bar(opacity=opacity).encode(
   x=alt.X("CRASH WEEKDAY:O", axis=alt.Axis(labelAngle=-30, title=None), sort=weekdayorder),
   xOffset="MOMENT:O",
   y=alt.Y("counts:Q", axis=alt.Axis(title="Collisions / Means", grid=True), scale=alt.Scale(domain=[0, 13000])),
   color=alt.Color("MOMENT:O", scale=alt.Scale(domain=list(colors.keys()), range=list(colors.values())))
).properties(title=alt.Title("Weekdays", fontSize=10, fontWeight=600))

averages_weekday = alt.Chart(weekdays_df).mark_rule(opacity=1).encode(
    y="mean(counts):Q",
    size=alt.value(2),
    color=alt.Color("MOMENT:O")
)


weekends_ch = alt.Chart(weekends_df).mark_bar(opacity=opacity).encode(
   x=alt.X("CRASH WEEKDAY:O", axis=alt.Axis(labelAngle=-30, title=None), sort=weekdayorder),
   xOffset="MOMENT:O",
   y=alt.Y(
       "counts:Q",
       axis=alt.Axis(title=None, labels=False, domain=False, ticks=False, grid=True),
       scale=alt.Scale(domain=[0, 13000])
   ),
   color=alt.Color(
       "MOMENT:O",
       scale=alt.Scale(domain=list(colors.keys()), range=list(colors.values())),
       legend=alt.Legend(title=None)
   )
).properties(title=alt.Title("Weekends", fontSize=10, fontWeight=600))

averages_weekend = alt.Chart(weekends_df).mark_rule(opacity=1).encode(
    y="mean(counts):Q",
    size=alt.value(2),
    color="MOMENT:O"
)



q1 = ((weekdays_ch + averages_weekday) | (weekends_ch + averages_weekend))

q1.configure_legend(symbolOpacity=1)

### 2. Is there any type of vehicle more prone to participate in accidents?
Obviously, with the current data we have this is impossible, as cars are the most predominant vehicle by a large margin, meaning they will have the most collisions. Lets start off viewing this data with a simle bar plot.

In [ ]:
vehicles = collisions.groupby(["VEHICLE"]).size().reset_index(name="counts")

alt.Chart(vehicles).mark_bar().encode(
    y=alt.Y("counts:Q", axis=alt.Axis(title="Collisions")),
    x=alt.X("VEHICLE:O", axis=alt.Axis(title=None, labelAngle=-30))
).properties(
    width=400
)

This confirms what we hypothesized earlier.

La primera idea que vam tenir va ser la de fer un paralel coordinate plane, on tinguessim els seguents plans:
- Percentatge d'accidents
- Percentatge de circulació
- Percentatge de ferits
- Percentatge de morts
- Ratio de ferits/accident
- Ratio de ferits/mort

Però a les dades proporcioandes no disposem del percentatge de circulació de cada vehicle i buscant per internet no hem trobat cap dataset que ens pugui proporcionar aquesta informació. Ara mirarem com es distribueixen els seguents plans:

In [ ]:
vehicles = collisions[["VEHICLE","NUMBER OF PERSONS INJURED", "NUMBER OF PERSONS KILLED"]]
vehicles = vehicles[vehicles["VEHICLE"] != "Unknown"]

vehicles = vehicles.groupby("VEHICLE").agg({
    "VEHICLE": "count",
    "NUMBER OF PERSONS INJURED": "sum",
    "NUMBER OF PERSONS KILLED": "sum"
}).rename(columns={"VEHICLE": "COLLISIONS"}).reset_index()

total_collisions = vehicles["COLLISIONS"].sum()

# Calcular el número de accidentes por tipo de vehículo
vehicles["% COLLISIONS"] = vehicles["COLLISIONS"] / total_collisions * 100

# Calcular el número total de personas heridas y muertas en todos los accidentes
total_injured = vehicles["NUMBER OF PERSONS INJURED"].sum()
total_killed = vehicles["NUMBER OF PERSONS KILLED"].sum()

# Calcular los porcentajes de personas heridas y muertas para cada tipo de vehículo
vehicles["% INJURED"] = vehicles["NUMBER OF PERSONS INJURED"] / total_injured * 100
vehicles["% KILLED"] = vehicles["NUMBER OF PERSONS KILLED"] / total_killed * 100

# Calcular los ratios de personas heridas y muertas por accidente para cada tipo de vehículo
vehicles["INJURED PER COLLISION"] = vehicles["NUMBER OF PERSONS INJURED"] / vehicles["COLLISIONS"]
vehicles["KILLED PER COLLISION"] = vehicles["NUMBER OF PERSONS KILLED"] / vehicles["COLLISIONS"]

vehicles.head()

In [ ]:
base = alt.Chart(vehicles, width=800).transform_window(
    index="count()"
).transform_fold(
    ["% COLLISIONS", "INJURED PER COLLISION", "KILLED PER COLLISION"]
).transform_joinaggregate(
    min="min(value)",
    max="max(value)",
    groupby=["key"]
).transform_calculate(
    norm_val="(datum.value - datum.min) / (datum.max - datum.min)",
    mid="(datum.min + datum.max) / 2"
)

lines = base.mark_line(opacity=0.3).encode(
    x="key:N",
    y= alt.Y("norm_val:Q", axis=None),
    color="VEHICLE:N",
    detail="index:N",
    opacity=alt.value(0.5)
)

rules = base.mark_rule(
    color="#ccc", tooltip=None
).encode(
    x="key:N",
    detail="count():Q",
) 

def ytick(yvalue, field):
    scale = base.encode(x="key:N", y=alt.value(yvalue), text=f"min({field}):Q")
    return alt.layer(
        scale.mark_text(baseline="middle", align="right", dx=-5, tooltip=None),
        scale.mark_tick(size=8, color="#ccc", orient="horizontal", tooltip=None)
    )

alt.layer(
    lines, rules ,ytick(0, "max"), ytick(150, "mid"), ytick(300, "min")
).configure_axisX(
    domain=False, labelAngle=0, tickColor="#ccc", title=None
).configure_view(
    stroke=None
)

Lets now try a scatter plot.

In [ ]:
maximum = max(vehicles["COLLISIONS"])
minimum = min(vehicles["COLLISIONS"])
mean = vehicles["COLLISIONS"].mean()

# Using purple color as it represents the entire collision count
scatter = alt.Chart(vehicles).mark_circle(color=colors[all_time]).encode(
    x=alt.X("INJURED PER COLLISION:Q", axis=alt.Axis(title="Injured per collision")),
    y=alt.Y("KILLED PER COLLISION:Q", axis=alt.Axis(title="Deaths per collision")),
    size=alt.Size("COLLISIONS:Q", scale=alt.Scale(range=[10, 700]), legend=alt.Legend(title="Total collisions (min-mean-max)", values=[minimum, mean, maximum])),
).properties(
    width=500,
    height=300
)

# Lets add labels for each vehicle
labels = scatter.mark_text(
    align="right",
    dx=-15,
    dy=0
).encode(
    text="VEHICLE:N",
    size=alt.value(10)
)

q2 = scatter + labels

q2

This one seems to be easier to understand and also looks nicer, we have decided to keep this one.

In [ ]:
(q1 & q2).configure_legend(symbolOpacity=1).resolve_scale(size="independent")

### 3. At what time of the day are accidents more common?
Lets make a simpler historgram with the overall average as well as a little mark indicating the max hour.

In [ ]:
time_df = collisions
time_df["HOUR"] = time_df["CRASH DATETIME"].dt.hour
time_df = time_df.groupby(["HOUR", "AFTER COVID"]).size().reset_index(name="counts")

time_df["MOMENT"] = np.where(time_df["AFTER COVID"], after, before)

time_ch = alt.Chart(time_df).mark_bar(opacity=opacity).encode(
    x=alt.X("HOUR:O", axis=alt.Axis(labelAngle=0), title="Hour"),
    y=alt.Y("counts:Q", title="Collisions / Mean"),
    color=alt.Color(
        "MOMENT:O",
        scale=alt.Scale(domain=list(colors.keys()), range=list(colors.values())),
        legend=alt.Legend(title=None)
    ),
    order=alt.Order("MOMENT:O", sort="ascending")
)

time_all_df = time_df.groupby(["HOUR"]).sum().reset_index()

averages_weekend = alt.Chart(time_all_df).mark_rule(opacity=1, color=colors[all_time]).encode(
    y="mean(counts):Q",
    size=alt.value(2),
)

max_hour = alt.Chart().mark_text(text=str(sum(time_df.loc[time_df["HOUR"] == 16, "counts"])), angle=0).encode(
    x=alt.value(330),
    y=alt.value(20),
)

q3 = (time_ch + averages_weekend + max_hour)
q3

In [ ]:
((q1 | q3) & q2).configure_legend(symbolOpacity=1).resolve_scale(size="independent")

### 4. Are there any areas with a larger number of accidents?
Lets make a choropleth map. First, lets just a couple collisions in NYC. We are using a district map.

In [ ]:
base = alt.Chart(map_data).mark_geoshape(fill="lightgray", stroke="black").project(type="albersUsa").properties(
    width=700,
    height=700
)

pts = alt.Chart(collisions[collisions["LOCATION"].notna()].head(5000)).mark_circle().encode(
    latitude="LATITUDE",
    longitude="LONGITUDE",
    color='BOROUGH',
    tooltip=['LATITUDE', "LONGITUDE"]
)

(base + pts)

Now making the Choropleth Map! We will be using the purple scale as we will be using the entire dataset, not just before/after covid. Keep in mind that we will only be looking at area, there are other factors too, like total km of streets. However, we have decided to go with this path as any other variable would be tricky to use.

In [ ]:
base = alt.Chart(map_data).mark_geoshape().project(type="albersUsa").encode(
    color=alt.Color("collision_count:Q", scale=alt.Scale(scheme='purples'), legend=alt.Legend(title="Collisions")),
).properties(
    width=600,
    height=600,
    title="NYC Community Districts"
)

base

Lets add labels to the top 3 areas with most collisions. Only 3 as getting too many more would overcrowd the map. Getting the labels from [here](https://furmancenter.org/files/sotc/SOC2007_IndexofCommunityDistricts_000.pdf). Using the centroids of the areas to get where to place the labels. Lets see how that looks. 

In [ ]:
top = map_data.sort_values(by="collision_count", ascending=False).head(3)
top[["LATITUDE", "LONGITUDE"]] = top["geometry"].centroid.apply(lambda x: pd.Series([x.y, x.x]))

# 
labels = {
    "boro_cd": ["412", "413", "305"],
    "LABELS": ["Jamaica / Hollis", "Queens Village", "East New York"]
}

top = top.merge(pd.DataFrame(labels), left_on="boro_cd", right_on="boro_cd")

text_labels = alt.Chart(top).mark_text(angle=0, dx=0, dy=0, fill="white", size=8).encode(
    longitude='LONGITUDE:Q',
    latitude='LATITUDE:Q',
    text='LABELS:N',
)

base + text_labels

Labels are good except the Queens Village, which is barely visible. Lets place it where it can be read correctly. And lets add a couple icons for "interesting vehicles"!. These icons will be wherever they collided!

In [ ]:
top.loc[top["LABELS"] == "Queens Village", ["LATITUDE", "LONGITUDE"]] = [40.66605, -73.75998]


text_labels = alt.Chart(top).mark_text(angle=0, dx=0, dy=0, fill="white", size=9).encode(
    longitude="LONGITUDE:Q",
    latitude="LATITUDE:Q",
    text="LABELS:N",
)


horse = alt.Chart(collisions[collisions["ORIGINAL VEHICLE"] == "Horse"]).mark_text(text="🐎", size=18).encode(
    longitude="LONGITUDE:Q",
    latitude="LATITUDE:Q",
)

gokart = alt.Chart(collisions[collisions["ORIGINAL VEHICLE"] == "Go kart"]).mark_text(text="🏎️", size=18).encode(
    longitude="LONGITUDE:Q",
    latitude="LATITUDE:Q",
)


q4 = (base + text_labels + horse + gokart).properties(width=600, height=600)

q4

Great! Lets now put it all together.

In [ ]:
((q4 | (q1 & q3)) & q2).configure_legend(symbolOpacity=1).resolve_scale(size="independent", color="shared")

### 5.  Is there a correlation between weather conditions and accidents?

In [ ]:
# Read weather data
weather = pd.read_csv("./processed-data/weather.csv")

weather_corr = weather.drop(columns=["valid"]).corr()
weather_corr

In [ ]:
# reshape the data into a long format
corr_long = weather_corr.stack().reset_index()
corr_long.columns = ['x', 'y', 'value']

# create the heatmap
heatmap = alt.Chart(corr_long).mark_rect().encode(
    x='x:O',
    y='y:O',
    color='value:Q'
).properties(
    width=300,
    height=300
)

# add text to the heatmap
text = heatmap.mark_text(baseline='middle').encode(
    text=alt.Text('value:Q', format='.2f'),
    color=alt.condition(
        alt.datum.value > 0.5,
        alt.value('white'),
        alt.value('black')
    )
)

heatmap + text

A partir d'aquest heatmap podem veure que hi ha una gran relació entre les columnes `vsby` i `relh`, els valors de visibilitat baixa están relacionats amb alts valors d'humitat relativa. També hi ha una gran relació entre les columnes `relh` i `tmpf`. Tot això, si ens parem a pensar amb els aspectes físics té molt sentit.

In [ ]:
# read the collision dataset
collisions['DATE'] = pd.to_datetime(collisions['CRASH DATETIME'])
weather['DATE'] = pd.to_datetime(weather['valid'])


# merge the two datasets on the common column "DATE"
collisions_weather = pd.merge(collisions, weather, on="DATE")

# print the merged dataset
print(collisions_weather.columns)

# select the columns we want to keep
collisions_weather_selected  = collisions_weather[['DATE', 'NUMBER OF PERSONS INJURED', 'NUMBER OF PERSONS KILLED', 'VEHICLE',  'tmpf', 'relh', 'sknt', 'p01i', 'vsby']]

alt.data_transformers.disable_max_rows()

In [ ]:
def violinPlot(dataset, column, rang):
    color = '#7fc97fbb' if dataset.equals(weather) else '#beaed4'
    title = 'Normal' if dataset.equals(weather) else 'Collisions'
    orient = 'right' if dataset.equals(weather) else 'left'
    chart = alt.Chart(dataset , width=100).transform_density(
        column,
        as_=[column, 'density'],
        extent= rang
    ).mark_area(orient='horizontal', color = color).encode(
        alt.X('density:Q')
            .stack('center')
            .impute(None)
            .title(None)
            .axis(labels=False, values=[0], grid=False, ticks=True),
        alt.Y(column + ':Q').title(title).axis(titleColor=color, orient=orient)
    )

    # Calculate quartiles
    q1 = dataset[column].quantile(0.25)
    q2 = dataset[column].quantile(0.5)
    q3 = dataset[column].quantile(0.75)

    # Add quartiles as horizontal lines
    q1_r = alt.Chart(pd.DataFrame({'y': [q1]})).mark_rule(color='#fee0d2', strokeWidth=2).encode(y='y')
    q2_r = alt.Chart(pd.DataFrame({'y': [q2]})).mark_rule(color='#fc9272', strokeWidth=2).encode(y='y')
    q3_r = alt.Chart(pd.DataFrame({'y': [q3]})).mark_rule(color='#de2d26', strokeWidth=2).encode(y='y')

    return chart + q1_r + q2_r + q3_r

(violinPlot(collisions_weather_selected, 'tmpf', [5, 45]) | 
 violinPlot(weather, 'tmpf', [5, 45])
).properties(
    title = "Temperature"
) | (violinPlot(collisions_weather_selected, 'relh', [0, 100]) | 
 violinPlot(weather, 'relh', [0, 100])
).properties(
    title = "Humidity"
) | (violinPlot(collisions_weather_selected, 'sknt', [0, 25]) | 
 violinPlot(weather, 'sknt', [0, 25])
).properties(
    title = "Speed of wind"
) | (violinPlot(collisions_weather_selected, 'p01i', [0, 0.5]) | 
 violinPlot(weather, 'p01i', [0, 0.5])
).properties(
    title = "Rainfall level"
) | (violinPlot(collisions_weather_selected, 'vsby', [0, 20]) | 
 violinPlot(weather, 'vsby', [0, 20])
).properties(
    title = "Visibility"
)


Amb aquest plot podem comparar la distribució de les variables climátiques quan hi ha accidents i aquestes mateixes en tot moment. La idea a l'hora de fer aquest gràfic era ajudar-nos a entendre si aquestes distribucions varien quan hi ha accidents, és a dir, si algunes de aquestes variables meteorologiques afecta en el nombre d'accidents.  
En alguns casos veiem distribucions lleugerament diferents però d'aquesta forma no ho podem comparar facilment. Hem de buscar una altra forma de respondre a la pregunta. La idea serà buscar els casos més extrems. Començarem amb la visibilitat, on clarament veiem que el primer quartil es troba a un nivell més baix quan hi ha accidents.

In [ ]:
print(f"Visibility in accidents: {collisions_weather_selected['vsby'].describe()}")
print(f"Visibility  in general: {weather['vsby'].describe()}")


A partir d'aquestes dades es pot concloure que quan la visibilitat és més baixa hi ha més accidents, doncs la mitjana així ho indica i el primer quartil també és més baix. Els altres quartils es troben en un valor de 16.093440 kilometres, que equival a 10 milles, valor considerat per la font de dades com una visibilitat complerta. 
Però caldria estudiar quina es la probabilitat d'accident amb visibilitat baixa enfront a la probabilitat d'accident amb visibilitat alta.

Una forma més clara de representar-ho seria la seguent: 
- Histograma on les X sigui la visibilitat y la Y el nombre d'accidents/vegades que apareix en weather

D'aquesta forma podem saber el ratio de colisions, que ens donarà una informació més valiosa sobre 

In [ ]:
# create 17 bins for the vsby column
bins = pd.cut(collisions_weather_selected.dropna(subset=["vsby"])["vsby"], bins=17, labels=list(range(17)))

# group by the bins
grouped = collisions_weather_selected.groupby(bins)

# get the count of collisions in each bin
counts = grouped.size()


# create 17 bins for the vsby column
bins_weather = pd.cut(weather.dropna(subset=["vsby"])['vsby'], bins=17, labels=range(17))

# group by the bins
grouped_weather = weather.groupby(bins_weather)

# get the count of collisions in each bin
counts_weather = grouped_weather.size()

In [ ]:
# create a dataframe with counts and counts_weather
df = pd.DataFrame({'counts': counts, 'counts_weather': counts_weather})

# create a new column with the ratio of counts to counts_weather
df['ratio'] = df['counts'] / df['counts_weather']

df['visibility'] = df.index 

# create the bar chart
chart = alt.Chart(df).mark_bar().encode(
    x=alt.X('visibility:O', axis=alt.Axis(title='Visibility')),
    y=alt.Y('ratio:Q', axis=alt.Axis(title='Ratio of Collisions')),
).properties(
    width=400,
    height=300
)



# add the mean to the plot
mean_line = alt.Chart(df).mark_rule(color='red', strokeDash=[5,5]).encode(
    y='mean(ratio):Q'
)

chart + mean_line

In [ ]:
# calculate the mean and standard deviation of the ratio
mean_ratio = df['ratio'].mean()
std_ratio = df['ratio'].std()

# calculate the Z-Score for each data point
df['z_score'] = (df['ratio'] - mean_ratio) / std_ratio

# create the scatter plot
scatter = alt.Chart(df).mark_circle().encode(
    x=alt.X('visibility:O', axis=alt.Axis(title='Visibility')),
    y=alt.Y('z_score:Q', axis=alt.Axis(title='Z-Score of Ratio')),
    color=alt.Color('z_score:Q', scale=alt.Scale(scheme='purplegreen')),
    tooltip=['visibility', 'z_score']
).properties(
    width=400,
    height=300
)

# add the mean line to the plot
mean_line = alt.Chart(df).mark_rule(color='red', strokeDash=[5,5]).encode(
    y='mean(z_score):Q'
)

scatter + mean_line

El que podem fer es agrupar els dos gràfics i tenim les des següents solucions:

In [ ]:
# create a dataframe with counts and counts_weather
df = pd.DataFrame({'counts': counts, 'counts_weather': counts_weather})

# create a new column with the ratio of counts to counts_weather
df['ratio'] = df['counts'] / df['counts_weather']

df['visibility'] = df.index 

mean_ratio = df['ratio'].mean()
std_ratio = df['ratio'].std()

df['pstd'] = mean_ratio + 2*std_ratio
df['nstd'] = mean_ratio - 2*std_ratio

# calculate the Z-Score for each data point
df['z_score'] = (df['ratio'] - mean_ratio) / std_ratio

# create the bar chart
bar = alt.Chart(df).mark_bar().encode(
    x=alt.X('visibility:O', axis=alt.Axis(title='Visibility')),
    y=alt.Y('ratio:Q', axis=alt.Axis(title='Ratio of Collisions')),
    color=alt.Color('z_score:Q', scale=alt.Scale(scheme='purplegreen'), legend = alt.Legend(title='Z-Score of Ratio'))
).properties(
    width=400,
    height=300
)

# create the bar chart
rule = alt.Chart(df).mark_rule().encode(
    x=alt.X('visibility:O', axis=alt.Axis(title='Visibility')),
    y=alt.Y('ratio:Q', axis=alt.Axis(title='Ratio of Collisions')),
    color=alt.Color('z_score:Q', scale=alt.Scale(scheme='purplegreen'), legend = None)
).properties(
    width=400,
    height=300
)

# create the bar chart
point = alt.Chart(df).mark_circle().encode(
    x=alt.X('visibility:O', axis=alt.Axis(title='Visibility')),
    y=alt.Y('ratio:Q', axis=alt.Axis(title='Ratio of Collisions')),
    color=alt.Color('z_score:O', scale=alt.Scale(scheme='purplegreen'), legend= None)
).properties(
    width=400,
    height=300
)

# add the mean to the plot
mean_line = alt.Chart(df).mark_rule(color='gray', strokeDash=[5,5]).encode(
    y='mean(ratio):Q'
)

pstd_line = alt.Chart(df).mark_rule(color='black', strokeDash=[5,5]).encode(
    y='pstd:Q'
)

nstd_line = alt.Chart(df).mark_rule(color='black', strokeDash=[5,5]).encode(
    y='nstd:Q'
)

(bar + mean_line + pstd_line + nstd_line) | (rule + point + mean_line + pstd_line + nstd_line)

Cap dels punts te un Z-Score superior a 2 per tant no podem extreure conclusions molt rellevants, però encara així podem veure que quan hi ha visibilitat practicament nula, ens trobem al voltant de 1.5 desviacions estandars i per visibilitat total ens trobem per sota de una desviació estandar. 

Perque els resultats son menys concluyents dels esperats? doncs perque la visibilitat està molt relacionada amb la humitat i aquesta amb la temperatura, que a la vegada està molt relacionada amb l'hora del dia. Per tant podem imaginar que les hores que hi ha menys visibilittat, coincideixen amb les hores en les quals hi ha menys accidents. Cosa que comprobarem seguidament. 

In [ ]:
# extract the hour from the DATE column
collisions_weather_selected['HOUR'] = collisions_weather_selected['DATE'].dt.hour

# group by hour and calculate the mean of the visibility column
mean_visibility = collisions_weather_selected.groupby('HOUR')['vsby'].mean()


# create a chart with the mean visibility by hour
visby_hour = alt.Chart(mean_visibility.reset_index()).mark_bar().encode(
    x=alt.X('HOUR:O', axis=alt.Axis(title='Hour')),
    y=alt.Y('vsby:Q', axis=alt.Axis(title='Mean Visibility')),
).properties(
    width=400,
    height=300
)

# add the mean line
mean_line = alt.Chart(mean_visibility.reset_index()).mark_rule(color='red', strokeDash=[5,5]).encode(
    y='mean(vsby):Q'
)

visby_hour + mean_line

Podem veure que entre les 6 i les 7 son les hores on menys visibilitat hi ha, que coincideixen amb hores amb poques colisions. Això no es una relació de causalitat, es a dir, no perque hi hagi menys visibilitat hi ha menys accidents, això no tindria sentit, sinó que just les hores on hi ha menys visibilitat, son en les que menys gent condueix i per tant menys accidents hi ha. Ens falten dades, però probablement sí que hi hauria més posibilitat d'accidents en aquestes hores.

In [ ]:
# create a new column with the visibility category
collisions_weather_selected['VISIBILITY CATEGORY'] = np.where(collisions_weather_selected['vsby'] > 16, 'High Visibility', 'Low Visibility')

# group by hour and visibility category and calculate the count of collisions
hourly_visibility = collisions_weather_selected.groupby(['HOUR', 'VISIBILITY CATEGORY']).size().reset_index(name='counts')


# calculate the total number of collisions per hour
hourly_total = hourly_visibility.groupby('HOUR')['counts'].sum().reset_index(name='total')

# merge the hourly_visibility and hourly_total dataframes
hourly_visibility = pd.merge(hourly_visibility, hourly_total, on='HOUR')

# calculate the percentage of low and high visibility collisions
hourly_visibility['percentage'] = hourly_visibility['counts'] / hourly_visibility['total'] * 100

# create the stacked bar chart
stacked_bar = alt.Chart(hourly_visibility).mark_bar().encode(
    x=alt.X('HOUR:O', axis=alt.Axis(title='Hour')),
    y=alt.Y('percentage:Q', axis=alt.Axis(title='Percentage of Collisions')),
    color=alt.Color('VISIBILITY CATEGORY:N', scale=alt.Scale(domain=['Low Visibility', 'High Visibility'], range=['#1f77b4', '#ff7f0e']), legend=alt.Legend(title='Visibility Category'))
).properties(
    width=400,
    height=300
)

stacked_bar

Lets now do the same with rainfall.

In [ ]:

# select the rows where p01i is 0
zero_p01i = collisions_weather_selected.loc[collisions_weather_selected['p01i'] == 0]

# get the number of rows with p01i = 0
num_zero_p01i = len(zero_p01i)

# select the rows where p01i is not 0
nonzero_p01i = collisions_weather_selected.loc[collisions_weather_selected['p01i'] != 0]

# create 10 bins for the p01i column
bins = pd.cut(nonzero_p01i['p01i'], bins=10)

# get the midpoint of each interval
midpoints = bins.apply(lambda x: x.mid.round(2))

# group by the midpoints
grouped = nonzero_p01i.groupby(midpoints)

# convert the result of the groupby to a dataframe
grouped_df = grouped.size().reset_index(name='counts')

# create a new dataframe with the count of rows with p01i = 0
zero_row = pd.DataFrame({'p01i': [0], 'counts': [num_zero_p01i]})

counts = pd.merge(zero_row , grouped_df, on=['p01i', 'counts'], how="outer", indicator=False)

# select the rows where p01i is 0
zero_p01i_weather = weather.loc[weather['p01i'] == 0]

# get the number of rows with p01i = 0
num_zero_p01i_weather = len(zero_p01i_weather)

# select the rows where p01i is not 0
nonzero_p01i_weather = weather.loc[weather['p01i'] != 0]

# create 10 bins for the p01i column
bins_weather = pd.cut(nonzero_p01i_weather['p01i'], bins=10)

# get the midpoint of each interval
midpoints_weather = bins_weather.apply(lambda x: x.mid.round(2))

# group by the midpoints
grouped_weather = nonzero_p01i_weather.groupby(midpoints_weather)

# convert the result of the groupby to a dataframe
grouped_df_weather = grouped_weather.size().reset_index(name='counts')

# create a new dataframe with the count of rows with p01i = 0
zero_row_weather = pd.DataFrame({'p01i': [0], 'counts': [num_zero_p01i_weather]})

counts_weather= pd.merge(zero_row_weather, grouped_df_weather, on=['p01i', 'counts'], how="outer", indicator=False)

In [ ]:
# create a dataframe with counts and counts_weather
df = pd.DataFrame({'p01i': counts['p01i'] ,'counts': counts['counts'], 'counts_weather': counts_weather['counts']})

# create a new column with the ratio of counts to counts_weather
df['ratio'] = df['counts'] / df['counts_weather']

mean_ratio = df['ratio'].mean()
std_ratio = df['ratio'].std()

df['mean'] = mean_ratio
df['pstd'] = mean_ratio + 2*std_ratio
df['nstd'] = mean_ratio - 2*std_ratio

# calculate the Z-Score for each data point
df['z_score'] = (df['ratio'] - mean_ratio) / std_ratio

df.fillna(0, inplace=True)

# create the bar chart
bar = alt.Chart(df).mark_bar().encode(
    x=alt.X('p01i:O', axis=alt.Axis(title='Rain level')),
    y=alt.Y('ratio:Q', axis=alt.Axis(title='Ratio of Collisions')),
    color=alt.Color('z_score:Q', scale=alt.Scale(scheme='purplegreen'), legend = alt.Legend(title='Z-Score of Ratio'))
).properties(
    width=400,
    height=300
)

# create the bar chart
rule = alt.Chart(df).mark_rule().encode(
    x=alt.X('p01i:O', axis=alt.Axis(title='Rain level')),
    y=alt.Y('ratio:Q', axis=alt.Axis(title='Ratio of Collisions')),
    color=alt.Color('z_score:Q', scale=alt.Scale(scheme='purplegreen'), legend = None)
).properties(
    width=400,
    height=300
)

# create the bar chart
point = alt.Chart(df).mark_circle().encode(
    x=alt.X('p01i:O', axis=alt.Axis(title='Rain Level')),
    y=alt.Y('ratio:Q', axis=alt.Axis(title='Ratio of Collisions')),
    color=alt.Color('z_score:O', scale=alt.Scale(scheme='purplegreen'), legend= None)
).properties(
    width=400,
    height=300
)

# add the mean to the plot
mean_line = alt.Chart(df).mark_rule(color='gray', strokeDash=[5,5]).encode(
    y='mean:Q'
)

pstd_line = alt.Chart(df).mark_rule(color='black', strokeDash=[5,5]).encode(
    y='pstd:Q'
)

nstd_line = alt.Chart(df).mark_rule(color='black', strokeDash=[5,5]).encode(
    y='nstd:Q'
)

(bar + mean_line + pstd_line + nstd_line) | (rule + point + mean_line + pstd_line + nstd_line)

El resultat que podriem esperar és que el ratio de colisions puja a mesura que aumenta el nivell de la pluja, i això es així fins a valors de 2.69 cm de pluja. Exceptuant la barra de 2.28 doncs sembla que tenim un valor atípic, doncs només ha plogut un cop en aquell interval. També es sorprenen que per a 3.93 torni a baixar, però de nou es un valor atípic que només ha passat un cop. Podriem com a solució plantejar el gràfic seguuent, plou vs no plou

In [ ]:
# select the rows where p01i is 0
zero_p01i = collisions_weather_selected.loc[collisions_weather_selected['p01i'] == 0]

# get the number of rows with p01i = 0
num_zero_p01i = len(zero_p01i)

# select the rows where p01i is not 0
nonzero_p01i = collisions_weather_selected.loc[collisions_weather_selected['p01i'] != 0]

# get the number of rows with p01i = 0
num_nonzero_p01i = len(nonzero_p01i)

# create a new dataframe with the count of rows with p01i = 0
zero_row = pd.DataFrame({'p01i': [0], 'counts': [num_zero_p01i]})

non_zero_row = pd.DataFrame({'p01i': [1], 'counts': [num_nonzero_p01i]})

counts = pd.merge(zero_row , non_zero_row, on=['p01i', 'counts'], how="outer", indicator=False)

# select the rows where p01i is 0
zero_p01i_weather = weather.loc[weather['p01i'] == 0]

# get the number of rows with p01i = 0
num_zero_p01i_weather = len(zero_p01i_weather)

# select the rows where p01i is not 0
nonzero_p01i_weather = weather[weather['p01i'] != 0]

# get the number of rows with p01i = 0
num_nonzero_p01i_weather = len(nonzero_p01i_weather)

# create a new dataframe with the count of rows with p01i = 0
zero_row_weather = pd.DataFrame({'p01i': [0], 'counts': [num_zero_p01i_weather]})

non_zero_row_weather = pd.DataFrame({'p01i': [1], 'counts': [num_nonzero_p01i_weather]})

counts_weather = pd.merge(zero_row_weather , non_zero_row_weather, on=['p01i', 'counts'], how="outer", indicator=False)

print(counts)

In [ ]:
# create a dataframe with counts and counts_weather
df = pd.DataFrame({'p01i': counts['p01i'] ,'counts': counts['counts'], 'counts_weather': counts_weather['counts']})

# create a new column with the ratio of counts to counts_weather
df['ratio'] = df['counts'] / df['counts_weather']

# create the bar chart
bar = alt.Chart(df).mark_bar().encode(
    x=alt.X('p01i:O', axis=alt.Axis(title='Rain level')),
    y=alt.Y('ratio:Q', axis=alt.Axis(title='Ratio of Collisions')),
).properties(
    width=400,
    height=300
)

bar 